In [5]:
import sys
sys.path.append("..")

import os
import json
import glob
import cv2
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns

from PIL import Image, ImageOps
from skimage import io
from skimage.color import rgba2rgb, rgb2xyz
from tqdm import tqdm
from dataclasses import dataclass
from math import floor, ceil

from main import calibrate_magnetic_wifi_ibeacon_to_position
from main import extract_magnetic_strength
from io_f import read_data_file
from visualize_f import visualize_trajectory, visualize_heatmap
from main import calibrate_magnetic_wifi_ibeacon_to_position
from main import extract_magnetic_strength
from main import extract_wifi_rssi, extract_wifi_count
from processing import SUBMISSION_SITES, split_sample_sbm_to_dataframe, formatting_for_visualize_waypoint,scrape_building_name
from tqdm import tqdm
import os

import lightgbm as lgb

sns.set_theme()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
test_paths = glob.glob("../test/*.txt")

In [8]:
wifi_df = []
for p in tqdm(test_paths):
    sample_test_path = p
    site = scrape_building_name(p)
    sample_data = read_data_file(sample_test_path)
    sample_wifi = sample_data.wifi
    for val in sample_wifi:
        tmp = [site]
        tmp.append(val[0])
        tmp.append(val[-1])
        wifi_df.append(tmp)
wifi_df = pd.DataFrame(wifi_df)
wifi_df.columns = ["site", "timestamp", "last_time_seen"]
# sampling = np.arange(0,len(wifi_ls),1)
# fig = plt.figure(figsize=(20,20))
display(wifi_df)


100%|██████████| 626/626 [02:11<00:00,  4.76it/s]


,site,timestamp,last_time_seen
0,5da958dd46f8266d0737457b,0000000002093,1571969331890
1,5da958dd46f8266d0737457b,0000000002093,1571969315536
2,5da958dd46f8266d0737457b,0000000002093,1571969315709
3,5da958dd46f8266d0737457b,0000000002093,1571969331090
4,5da958dd46f8266d0737457b,0000000002093,1571969330837
...,...,...,...
6305384,5d27096c03f801723c31e5e0,0000000106860,1571890225204
6305385,5d27096c03f801723c31e5e0,0000000106860,1571890226590
6305386,5d27096c03f801723c31e5e0,0000000106860,1571890215115
6305387,5d27096c03f801723c31e5e0,0000000106860,1571890226631


In [9]:
wifi_df.sort_values("last_time_seen")

,site,timestamp,last_time_seen
4930520,5a0546857ecc773753327266,0000000002066,1561434808031
4930446,5a0546857ecc773753327266,0000000002066,1561434808350
4930441,5a0546857ecc773753327266,0000000002066,1561434808350
4930454,5a0546857ecc773753327266,0000000002066,1561434808423
4930518,5a0546857ecc773753327266,0000000002066,1561434808622
...,...,...,...
4469976,5a0546857ecc773753327266,0000000065925,1578483023019
4469975,5a0546857ecc773753327266,0000000065925,1578483023019
4469981,5a0546857ecc773753327266,0000000065925,1578483023020
4469977,5a0546857ecc773753327266,0000000065925,1578483023021
